In [1]:
#importando bibliotecas e dando LOAD no template
import pandas as pd
import numpy as np

from dash import html, dcc, Dash
from dash.dependencies import Input, Output
import plotly.express as px
from dash.exceptions import PreventUpdate

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from IPython.display import Image, display

load_figure_template("FLATLY")

In [2]:
#importando as bases de dados
calendar_df = pd.read_csv("MavenMarket_Calendar.csv")
customers_df = pd.read_csv("MavenMarket_Customers.csv")
products_df = pd.read_csv("MavenMarket_Products.csv")
regions_df = pd.read_csv("MavenMarket_Regions.csv")
returns_df = pd.read_csv("MavenMarket_Returns_1997-1998.csv")
stores_df = pd.read_csv("MavenMarket_Stores.csv")
transactions_df = pd.read_csv("MavenMarket_Transactions.csv")

In [3]:
#Renomear nome das lojas
stores_df['store_name'] = stores_df['store_name'].map({'Store 1': 'Acapulco Store', 'Store 2': 'Bellingham Store','Store 3':'Bremerton Store'
                                                      ,'Store 4':'Camacho','Store 5':'Guadalajara Store','Store 6':'Beverly Hills Store'
                                                      ,'Store 7':'Los Angeles Store','Store 8':'Merida Store','Store 9':'Mexico City Store'
                                                      ,'Store 10':'Orizaba Store','Store 11':'Portland Store','Store 12':'Hidalgo Westchester Store'
                                                        ,'Store 13':'Salem Store','Store 14':'San Francisco Store','Store 15':'Seattle Store'
                                                       ,'Store 16':'Spokane Store','Store 17':'Tacoma Store','Store 18':'Hidalgo Glen Road Store'
                                                       ,'Store 19':'Vancouver Store','Store 20':'Victoria Store','Store 21':'San Andres Store'
                                                       ,'Store 22':'Walla Walla Store','Store 23':'Yakima Store','Store 24':'San Diego Store'})

#Definindo 'date' como data e criando separações de meses, dia e ano
calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%m/%d/%Y')
returns_df['return_date'] = pd.to_datetime(returns_df['return_date'], format='%m/%d/%Y')
returns_df.rename(columns={'return_date': 'date'}, inplace=True)
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%m/%d/%Y')

# Crie colunas separadas para mês, dia e ano
calendar_df['Mês'] = calendar_df['date'].dt.month
calendar_df['Dia'] = calendar_df['date'].dt.day
calendar_df['Ano'] = calendar_df['date'].dt.year



In [4]:
#Mudando o nome da coluna de data em transactions para 'date'
transactions_df.rename(columns={'transaction_date': 'date'}, inplace=True)

#Dando Merge na transactions com o calendar e criando o DF completo que será usado
MM_df = pd.merge(transactions_df, calendar_df, on='date', how='left')

#Dando Merge na transactions com Products para calcular Receita e Lucro
MM_df = pd.merge(MM_df, products_df, on='product_id', how='left')


#Calculando Receita Total e o Lucro
MM_df['Receita Total'] = MM_df['quantity']*MM_df['product_retail_price']
MM_df['Lucro Total'] = MM_df['Receita Total'] - (MM_df['quantity']*MM_df['product_cost'])

#Fazendo a data por período de meses e transformando em str
MM_df['m_date'] = MM_df['date'].dt.to_period("M")
MM_df['date_str'] = MM_df['m_date'].dt.strftime('%Y-%m')



In [5]:
#Merge da planilha de regiões em stores
MM_stores_df = pd.merge(stores_df, regions_df, on='region_id', how='left')

#Merge na planilha de stores
MM_df_stores = pd.merge(MM_df, MM_stores_df, on='store_id', how='left')

#Merge da planilha Customers para calcular Ticket Médio por Cliente
MM_df_customers = pd.merge(MM_df,customers_df,on='customer_id',how='left')

In [6]:
#Merge da planilha de Retornos com a de data
MM_RET_df = pd.merge(returns_df, calendar_df, on='date', how='left')

#Merge Products e Returns para cálculo das devoluções em $
MM_RET_df = pd.merge(returns_df, products_df, on='product_id', how='left')

#Cálculo de total de retorno em $
MM_RET_df['Retorno Total'] = MM_RET_df['quantity']*MM_df['product_retail_price']

#Fazendo a data por período de meses e transformando em str
MM_RET_df['m_date'] = MM_RET_df['date'].dt.to_period("M")
MM_RET_df['date_str'] = MM_RET_df['m_date'].dt.strftime('%Y-%m')

In [7]:
#Criando a aplicação com o Tema FLATLY
app = Dash(__name__, external_stylesheets = [dbc.themes.FLATLY])


#Criação dos cartões de indicadores
Rec_card = dbc.Card(
    [
        html.H1('Receita Mensal', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='Receita_KPI', style={'font-size': '18px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Luc_card = dbc.Card(
    [
        html.H1('Lucro Total por Mês', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='Lucro_KPI', style={'font-size': '18px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Ret_card = dbc.Card(
    [
        html.H1('Retorno Mensal', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='Retorno_KPI', style={'font-size': '18px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Tck_card = dbc.Card(
    [
        html.H1('Ticket Médio', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='TCK', style={'font-size': '18px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Sqf_card = dbc.Card(
    [
        html.H1('Receita por sqft', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='KPI_sqft', style={'font-size': '18px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Brandtop1_card = dbc.Card(
    [
        html.H1('Maior Receita: Marca', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='top1brand', style={'font-size': '20px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

Producttop1_card = dbc.Card(
    [
        html.H1('Maior Receita: Produto', style={'font-size': '15px','color':'white','text-align':'center'}),
        html.H1(id='top1product', style={'font-size': '20px','color':'white','text-align':'center'})
    ], 
    color='primary'
)

#Layout da Aplicação-----------------------------------------------------------------------------------------------------

app.layout = html.Div([
    
    dcc.Tabs(id = 'Tabs', value='Sumário Exec.',
            children = [
                
                dcc.Tab(id = 'Tab1',value = 'Sumário Exec.', label='Sumário Exec.',
                        
                        children = [
    
    dbc.Row([
        
        dbc.Col([
            
    
    html.H4('Indicadores'), 
    
    # Indicador de Receita
    dbc.Row(
        dbc.Col([Rec_card, html.Br()]
        )
    ),
    
    # Indicador Lucro
    dbc.Row(
        dbc.Col([Luc_card, html.Br()]
        )
    ),
    
    # Indicador de Retorno
    dbc.Row(
        dbc.Col([Ret_card, html.Br()]
        )
    ),
    
    # Indicador de Ticket Médio
     dbc.Row(
        dbc.Col([Tck_card, html.Br()]
        )
    ),
    
    # Row5 - Quarto Indicador
    dbc.Row(
        dbc.Col([Sqf_card, html.Br()]
        )
    )], width = 2),
        
        
    #Coluna do Gráfico
        dbc.Col(
    
        dcc.Graph(id='Serie_Receita'), width = 10
        
        )]),
   
    
#Segunda Row-----------------------------------------------------------------------------------------------------------
    dbc.Row([
        
        dbc.Col([
            
            html.Br(),
        
            html.H4('Selecione as Lojas'),
            
            dcc.Checklist(id='Lojas', options = MM_df_stores['store_name'].unique(), value = 'Store 6',
                  style={'maxHeight': '280px', 'overflowY': 'scroll'})], width=3),
            
            dbc.Col(
        
        dcc.Dropdown(id='dropdown', options=MM_df['date_str'].unique(), value='1998-12'), width=2,
                ),
        
        dbc.Col(
            
            dcc.Graph(id='Bar_Regions'), width = 7
        )]),  
                
                  
                  
                            
                            
                            
                #Fechou a Tab
                        ]),
                
                
  #Segunda Tab -----------------------------------------------------------------------------------------------------------              
    dcc.Tab(
        label = 'Produtos', value = 'Tab2', children = [
            
            #Row1
            dbc.Row([
                
                dbc.Col([
                
                #Row1.1
                dbc.Row(
                    dbc.Col(dcc.Graph(id = 'Top3ven'), width=12)
                    ),
                
                #Row1.2
                dbc.Row(
                dbc.Col([Brandtop1_card, html.Br()],width=6)
                ),
                    ], width=5),
                
                dbc.Col(dcc.Graph(id = 'Rec_mar'), width=7),
                   
                
                
                #Fechou Row 1
                ]),
            
            
            #Row2
            dbc.Row([
                
                dbc.Col([
                
                #Row2.1
                dbc.Row(
                    dbc.Col([Producttop1_card,html.Br()], width=12)
                    ),
                
                #Row2.2
                dbc.Row([
                dbc.Col(dcc.Checklist(id='marcas',options=products_df['product_brand'].unique(),
                          style={'maxHeight': '280px', 'overflowY': 'scroll'}),width=6),
                
                    
                dbc.Col(dcc.DatePickerRange(id="datetab2",start_date=calendar_df['date'].min(),
                                            end_date=calendar_df['date'].max(),display_format="YYYY-MM"),
                        style={'maxHeight': '280px', 'overflowY': 'scroll'},width=6)
                ]),
                    ], width=5),
                
                dbc.Col(dcc.Graph(id = 'Ret_mar'), width=7),
                   
                
                
                #Fechou Row 2
                ]),   
            
            
            #Fechou Tab2   
        ])
                        
                #Fechou Tabs
            ]),
    
    
    #Fechou o html.Div
    ])
    



#----------------------------------------------------------------------------------------------------------------------
#Backend com a função Callback
@app.callback(
     [Output('Receita_KPI','children'), Output('Lucro_KPI','children'),
      Output('Serie_Receita','figure'),Output('Bar_Regions','figure'),
      Output('Retorno_KPI','children'),Output('TCK','children'),
      Output('KPI_sqft','children'), Output('Top3ven','figure'),
      Output('Ret_mar','figure'), Output('Rec_mar','figure'),
      Output('top1brand','children'), Output('top1product','children')],                             
     [Input('dropdown','value'), Input('Lojas','value'), 
      Input('datetab2','start_date'), Input('datetab2','end_date'),
      Input('marcas','value')]

)

#-----------------------------------------------------------------------------------------------------------------------
def data_kpi(date, loja, start, end, marcas):
    
    # Filtrar o DataFrame MM_df e MM_RET_df com base no mês e ano
    MM_df_filtrado = MM_df.loc[MM_df['date_str']==date]
    MM_RET_df_filtrado = MM_RET_df.loc[MM_RET_df['date_str']==date]
    
    # Calcular a soma da coluna 'Receita Total' para o mês e ano selecionados
    KPI_rec = MM_df_filtrado['Receita Total'].sum()
    
    #Calcular a soma da coluna 'Lucro Total' para o período selecionado
    KPI_luc = MM_df_filtrado['Lucro Total'].sum()
    
    #Calcular a soma da coluna 'Retorno Total' para o período selecionado
    KPI_ret = MM_RET_df_filtrado['Retorno Total'].sum().round(2)
    
    #Criando Ticket Médio por Cliente
    MM_df_customers_filt = MM_df_customers.loc[MM_df_customers['date_str']==date]
    
    Clientes_Unicos = MM_df_customers_filt['customer_id'].unique()
    Clientes_Total = len(Clientes_Unicos)
    ticket_medio_por_cliente = MM_df_customers_filt['Receita Total'].sum()/Clientes_Total
    
    #Criando Receita por Metro Quadrado
    df_sqft = MM_df_stores.loc[MM_df_stores['date_str'] == date]
    
    if '1998' in date:
        REC_sqft = df_sqft['Receita Total'].sum() / (stores_df['total_sqft'].sum() - 214446)
    else:
        REC_sqft = df_sqft['Receita Total'].sum() / stores_df['total_sqft'].sum()
    
    #Gráfico de linhas
    if not isinstance(loja, list):
        loja = [loja]
    df_loja = MM_df_stores[MM_df_stores['store_name'].isin(loja)]
    df_loja = df_loja.groupby('date_str', as_index=False).sum(numeric_only=True)

    # Criando um DataFrame para a soma total de receita de todas as lojas
    df_receita_total = MM_df.groupby('date_str', as_index=False)['Receita Total'].sum()

    # Criando segundo eixo y separado
    fig_subplots = make_subplots(specs=[[{"secondary_y": True}]])

    # Adicionando a linha da receita total de todas as lojas ao gráfico
    fig_subplots.add_trace(go.Scatter(x=df_receita_total['date_str'], y=df_receita_total['Receita Total'], mode='lines', name='Receita Total'))

    # Adicionando a linha da loja específica ao eixo secundário
    fig_subplots.add_trace(go.Scatter(x=df_loja['date_str'], y=df_loja['Receita Total'], mode='lines', name='Receita lojas', line=dict(dash='dash')), secondary_y=True)

    # Definindo rótulos dos eixos e título
    #fig_subplots.update_xaxes(title_text="Data")
    #fig_subplots.update_yaxes(title_text="Receita Total de Todas as Lojas", secondary_y=False)
    #fig_subplots.update_yaxes(title_text="Receita das Lojas", secondary_y=True)
    fig_subplots.update_layout(title="Receita Total e Receita da Loja Específica")
    
    
    
    
    #Gráfico de barras Receita x Região
    fig_bar = px.bar(MM_df_stores.groupby(MM_df_stores['sales_region'],as_index=False).sum(numeric_only=True),
                     x='sales_region',
                     y='Receita Total'
    )
    
    fig_bar.update_layout(title = 'Receita por Região')
    
    #Formatação dos números
    KPI_rec_formatted = f"${KPI_rec:,.2f}"
    KPI_luc_formatted = f"${KPI_luc:,.2f}"
    KPI_ret_formatted = f"${KPI_ret:,.2f}"
    ticket_medio_por_cliente_formatted = f"${ticket_medio_por_cliente:.2f}"
    REC_sqft_formatted = f"${REC_sqft:,.2f}"
    
    
#ACABOU SUMÁRIO EXECUTIVO, INICIANDO PRODUTOS ---------------------------------------------------------------------------

    #Top3 Produtos mais vendidos
    filtered_MM_df_top3 = MM_df_stores[MM_df_stores['date'].between(start,end)]
    MM_df_top3 = filtered_MM_df_top3.groupby('product_name',as_index=False).sum(numeric_only=True)
    MM_df_top3 = MM_df_top3.nlargest(3,'quantity')
    fig_top3 = px.pie(MM_df_top3,values='quantity',names='product_name')
    
    #Top 6 Retornos por Marcas
    df_brands_filtrado = MM_df_stores[MM_df_stores['date'].between(start,end)]
    MM_RET_filtrado = MM_RET_df[MM_RET_df['date'].between(start,end)]
    teste_agr = df_brands_filtrado.groupby('product_brand', as_index=False).sum(numeric_only=True)
    Receita_marc = teste_agr[['product_brand','Receita Total']]
    MM_RET_Porc = pd.merge(Receita_marc,MM_RET_filtrado, on='product_brand', how='left')
    MM_RET_Porc['RET%'] = MM_RET_Porc['Retorno Total']/MM_RET_Porc['Receita Total']
    MM_RET_brands = MM_RET_Porc.groupby('product_brand', as_index=False).sum(numeric_only=True)
    MM_RET_brands = MM_RET_brands.nlargest(6,'RET%')
    fig_RETmar = px.bar(MM_RET_brands,x='product_brand',y='RET%')
    
    #Série Temporal: Receita x Marca
    if not isinstance(marcas, list):
        marcas = [marcas]
    marcas_df = MM_df_stores[MM_df_stores['product_brand'].isin(marcas)]
    marcas_df = marcas_df.groupby('date_str', as_index=False).sum(numeric_only=True)
    ReceitaTotal_df = MM_df_stores.groupby('date_str', as_index=False)['Receita Total'].sum()

    fig_subplots_2 = make_subplots(specs=[[{"secondary_y": True}]])
    fig_subplots_2.add_trace(go.Scatter(x=ReceitaTotal_df['date_str'], y=ReceitaTotal_df['Receita Total'], mode='lines', name='Receita Total'))
    fig_subplots_2.add_trace(go.Scatter(x=marcas_df['date_str'], y=marcas_df['Receita Total'], mode='lines', name='Receita por Marcas', line=dict(dash='dash')), secondary_y=True)
    fig_subplots_2.update_layout(title="Receita Total e Receita por Marca")
    
    #Cards Marca com Maior Receita e Produto com Maior Receita
    
    Brandtop1_df = filtered_MM_df_top3.groupby('product_brand',as_index=False).sum(numeric_only=True)
    Brandtop1_df = Brandtop1_df.nlargest(1,'Receita Total')
    Brandtop_name = Brandtop1_df['product_brand']
    
    Producttop1_df = filtered_MM_df_top3.groupby('product_name',as_index=False).sum(numeric_only=True)
    Producttop1_df = Producttop1_df.nlargest(1,'Receita Total')
    Producttop_name = Producttop1_df['product_name']
    

    return (KPI_rec_formatted, KPI_luc_formatted, 
            fig_subplots,fig_bar,
            KPI_ret_formatted,ticket_medio_por_cliente_formatted,
            REC_sqft_formatted,fig_top3,fig_RETmar,fig_subplots_2, Brandtop_name,
            Producttop_name)



#Rodando a aplicação
if __name__ == "__main__":
    app.run_server()